In [39]:
import datetime
import os.path
from dateutil import parser as date_parser
from dotenv import load_dotenv;

# Google Libraries
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Apple (CalDAV) Libraries
import caldav
from icalendar import Calendar


In [40]:
load_dotenv()

USE_GOOGLE_CALENDAR = True
USE_APPLE_CALENDAR = True

In [41]:
GOOGLE_SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

APPLE_ID_EMAIL = os.getenv("APPLE_ID_EMAIL") 
APP_SPECIFIC_PASSWORD = os.getenv("APP_SPECIFIC_PASSWORD")
CALDAV_URL = "https://caldav.icloud.com"


In [42]:
def authenticate_google():
    """Authenticates with Google OAuth 2.0 and returns the credentials object."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", GOOGLE_SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", GOOGLE_SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return creds

    

def fetch_google_events(creds):
    """Fetches events from Google Calendar and converts them into a common format."""
    google_events = []
    try:
        service = build("calendar", "v3", credentials=creds)
        now = datetime.datetime.utcnow().isoformat() + "Z"
        events_result = service.events().list(
            calendarId="primary",
            timeMin=now,
            maxResults=20,
            singleEvents=True,
            orderBy="startTime"
        ).execute()
        events = events_result.get("items", [])

        for event in events:
            start_str = event["start"].get("dateTime", event["start"].get("date"))
            google_events.append({
                "summary": event["summary"],
                "start_time": date_parser.parse(start_str),
                "source": "Google"
            })
        print("Google Calendar events retrieved successfully.")
    except Exception as e:
        print(f"An error occurred while retrieving data from Google Calendar: {e}")
    return google_events

In [43]:
def fetch_apple_events(email, password, url):
    """Fetches events from Apple iCloud via CalDAV and converts them into a common format."""
    apple_events = []
    try:
        client = caldav.DAVClient(url=url, username=email, password=password)
        principal = client.principal()
        
        now = datetime.datetime.now()
        one_month_later = now + datetime.timedelta(days=30)
        
        for calendar in principal.calendars():
            results = calendar.date_search(start=now, end=one_month_later, expand=True)
            for event_raw in results:
                cal = Calendar.from_ical(event_raw.data)
                for component in cal.walk():
                    if component.name == "VEVENT":
                        apple_events.append({
                            "summary": component.get('summary'),
                            "start_time": component.get('dtstart').dt,
                            "source": "Apple"
                        })
        print("Apple Calendar events retrieved successfully.")
    except Exception as e:
        print(f"An error occurred while retrieving data from Apple Calendar: {e}")
    return apple_events


In [44]:
all_events = []


if USE_GOOGLE_CALENDAR:
    print("--- Connecting to Google Calendar ---")
    try:
        google_creds = authenticate_google()
        google_events = fetch_google_events(google_creds)
        all_events.extend(google_events)
    except Exception as e:
        print(f"Google authentication failed: {e}")


if USE_APPLE_CALENDAR:
    print("\n--- Connecting to Apple Calendar ---")
    apple_events = fetch_apple_events(APPLE_ID_EMAIL, APP_SPECIFIC_PASSWORD, CALDAV_URL)
    all_events.extend(apple_events)

if all_events:
    now_tz = datetime.datetime.now().astimezone().tzinfo
    for event in all_events:
        if event['start_time'].tzinfo is None:
            event['start_time'] = event['start_time'].replace(tzinfo=now_tz)

    all_events.sort(key=lambda x: x['start_time'])
    print("\nAll calendars have been merged and sorted.")

--- Connecting to Google Calendar ---


/var/folders/5y/q1w058gx5fq5_dhy09c75jfc0000gn/T/ipykernel_53397/1264591709.py:23: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + "Z"


Google Calendar events retrieved successfully.

--- Connecting to Apple Calendar ---


/var/folders/5y/q1w058gx5fq5_dhy09c75jfc0000gn/T/ipykernel_53397/2161321627.py:12: DeprecationWarning: use `calendar.search rather than `calendar.date_search`
  results = calendar.date_search(start=now, end=one_month_later, expand=True)


Apple Calendar events retrieved successfully.

All calendars have been merged and sorted.


In [45]:
if all_events:
    print("\n--- MERGED CALENDAR AGENDA ---")
    for event in all_events:
        source_tag = f"[{event['source']}]"
        
        local_time = event['start_time'].astimezone()
        
        print(f"🗓️  {local_time.strftime('%d %b %Y, %H:%M')} - {event['summary']} {source_tag}")
else:
    print("\nNo events found to display.")


--- MERGED CALENDAR AGENDA ---
🗓️  17 Aug 2025, 18:00 - Aaaa [Apple]
